In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install datasets rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64", but you 

In [3]:
from datasets import load_dataset
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize

# Example dataset (like "ag_news" or your own IR dataset)
dataset = load_dataset("ag_news", split="train[:1000]")
corpus = [item["text"] for item in dataset]

tokenized_corpus = [word_tokenize(doc.lower()) for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

# Sample query
query = "latest economic news"
tokenized_query = word_tokenize(query.lower())
results = bm25.get_top_n(tokenized_query, corpus, n=5)


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [4]:
from transformers import pipeline

# LLM-based zero-shot classifier (semantic NLU)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
label = "economic news"

for doc in results:
    print(classifier(doc, candidate_labels=[label]))


2025-04-23 09:19:44.385168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745399984.617916      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745399984.686429      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


{'sequence': "News: UK police issue 'vicious' Trojan alert Britain's top cybercrime fighters have joined up with the banking industry today in warning of the latest attempt to defraud online banking customers.\\", 'labels': ['economic news'], 'scores': [0.004376047756522894]}
{'sequence': 'Expansion slows in Japan Economic growth in Japan slows down as the country experiences a drop in domestic and corporate spending.', 'labels': ['economic news'], 'scores': [0.9980980157852173]}
{'sequence': 'Iranian economic reform falters The Iranian parliament votes against key parts of a reform plan aimed at opening the economy to foreign investment.', 'labels': ['economic news'], 'scores': [0.9959885478019714]}
{'sequence': 'Indonesia urges debt flexibility Indonesia calls on the International Monetary Fund to help reschedule its debts, as figures reveal a slowdown in economic growth.', 'labels': ['economic news'], 'scores': [0.9911000728607178]}
{'sequence': "Money Funds Fell in Latest Week (AP)